In [1]:
#Import packages

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
import statsmodels.api as sm
import statsmodels.stats.api as sms
from statsmodels.stats.diagnostic import het_white

In [2]:
#Import CSV's
path = r'/Applications/Desktop Folder/Data/'
b_data = pd.read_csv(path + 'Borrower.csv')
l_data = pd.read_csv(path + 'Loans.csv')

In [3]:
#Look at borrower dataset
b_data.head()

,Account_Number,First_Name,Last_Name,Zip_Code,Country_Code,Income_Annual,Total_Mortgage_Balances,Total_HE_Balances,Num_Trades_Past_Due,Num_HE_Trades
0,5000,Rebecca,Berry,44130.0,US,$0.00,0,0,NaN,NaN
1,4999,Connor,Mathis,48162.0,US,$0.00,0,133829,93.0,3.0
2,4998,Mary,Sanderson,46017.0,US,$0.00,0,81498,1.0,2.0
3,4997,Ryan,Vance,46052.0,US,$0.00,18691,23091,93.0,1.0
4,4996,Lillian,Sharp,45840.0,US,$0.00,0,13107,93.0,2.0


In [4]:
#Look at loan dataset
l_data.head()

,Account_Number,Month,Product,FICO_Score_Original,FICO_Score_Current,Origination_Date,Maturity_Date,Commitments,Outstandings,LTV_Original,LTV_Current,Lien_Position,Day_Past_Due,Interest_Variability_Code,Collateral_Value,Payment_Amount,Origination_DTI,APR
0,5000,31-Mar-17,HELOAN,677,0,24-Jul-12,21-Aug-22,0.0,0.0,0.8,0.137,JR_LIEN,0,FXD,$0.00,$0.00,0.3,0.02
1,5000,30-Apr-17,HELOAN,677,0,24-Jul-12,21-Aug-22,0.0,0.0,0.8,0.137,JR_LIEN,0,FXD,$0.00,$0.00,0.3,0.02
2,5000,31-May-17,HELOAN,677,0,24-Jul-12,21-Aug-22,0.0,0.0,0.8,0.136,JR_LIEN,0,FXD,$0.00,$0.00,0.3,0.02
3,5000,30-Jun-17,HELOAN,677,0,24-Jul-12,21-Aug-22,0.0,0.0,0.8,0.141,JR_LIEN,0,FXD,$0.00,$0.00,0.3,0.02
4,5000,31-Jul-17,HELOAN,677,0,24-Jul-12,21-Aug-22,0.0,0.0,0.8,0.140,JR_LIEN,0,FXD,$0.00,$0.00,0.3,0.02


In [5]:
#Merge datasets, drop unnecessary variables, check missing values, drop missing values
data = pd.merge(l_data, b_data, on="Account_Number")
data = data.drop(['First_Name','Last_Name','Maturity_Date','Origination_Date', 'Payment_Amount','Income_Annual','Country_Code','Collateral_Value'],axis=1)
print(data.isna().sum())
data = data.dropna()
data.head()

Account_Number                   0
Month                            0
Product                          0
FICO_Score_Original              0
FICO_Score_Current               0
Commitments                      0
Outstandings                     0
LTV_Original                     0
LTV_Current                      0
Lien_Position                    0
Day_Past_Due                     0
Interest_Variability_Code    10998
Origination_DTI                  0
APR                              0
Zip_Code                     11589
Total_Mortgage_Balances          0
Total_HE_Balances                0
Num_Trades_Past_Due          15726
Num_HE_Trades                15654
dtype: int64


,Account_Number,Month,Product,FICO_Score_Original,FICO_Score_Current,Commitments,Outstandings,LTV_Original,LTV_Current,Lien_Position,Day_Past_Due,Interest_Variability_Code,Origination_DTI,APR,Zip_Code,Total_Mortgage_Balances,Total_HE_Balances,Num_Trades_Past_Due,Num_HE_Trades
6,4999,31-Mar-17,HELOAN,840,722,129012.21,141234.42,0.8,0.490,SR_LIEN,0,FXD,0.2,0.04,48162.0,0,133829,93.0,3.0
7,4999,30-Apr-17,HELOAN,840,726,135560.48,130138.06,0.8,0.447,SR_LIEN,0,FXD,0.2,0.04,48162.0,0,133829,93.0,3.0
8,4999,31-May-17,HELOAN,840,712,139361.48,131243.34,0.8,0.447,SR_LIEN,0,FXD,0.2,0.04,48162.0,0,133829,93.0,3.0
9,4999,30-Jun-17,HELOAN,840,688,128305.87,128305.87,0.8,0.462,SR_LIEN,0,FXD,0.2,0.04,48162.0,0,133829,93.0,3.0
10,4999,31-Jul-17,HELOAN,840,675,141538.98,133451.04,0.8,0.549,SR_LIEN,0,FXD,0.2,0.04,48162.0,0,133829,93.0,3.0


In [6]:
#Create categorical identifiers for Outstandings and Commitments variables
data.loc[(data['Outstandings'].astype(int)==0) , 'Outstandings'] = 0
data.loc[(data['Outstandings'].astype(int) >0) & (data['Outstandings'].astype(int) <=50000), 'Outstandings'] = 1
data.loc[(data['Outstandings'].astype(int) >=50000 ) & (data['Outstandings'].astype(int) <=100000), 'Outstandings'] = 2
data.loc[(data['Outstandings'].astype(int) >=100000 ) & (data['Outstandings'].astype(int) <=200000), 'Outstandings'] = 3
data.loc[(data['Outstandings'].astype(int) >=200000 ) & (data['Outstandings'].astype(int) <=300000), 'Outstandings'] = 4
data.loc[(data['Outstandings'].astype(int) >=300000 ) & (data['Outstandings'].astype(int) <=400000), 'Outstandings'] = 5
data.loc[(data['Outstandings'].astype(int) >=400000 ), 'Outstandings'] = 6

data.loc[(data['Commitments'].astype(int)==0) , 'Commitments'] = 0
data.loc[(data['Commitments'].astype(int) >0) & (data['Commitments'].astype(int) <=50000), 'Commitments'] = 1
data.loc[(data['Commitments'].astype(int) >=50000 ) & (data['Commitments'].astype(int) <=100000), 'Commitments'] = 2
data.loc[(data['Commitments'].astype(int) >=100000 ) & (data['Commitments'].astype(int) <=200000), 'Commitments'] = 3
data.loc[(data['Commitments'].astype(int) >=200000 ) & (data['Commitments'].astype(int) <=300000), 'Commitments'] = 4
data.loc[(data['Commitments'].astype(int) >=300000 ) & (data['Commitments'].astype(int) <=400000), 'Commitments'] = 5
data.loc[(data['Commitments'].astype(int) >=400000 ), 'Commitments'] = 6

In [7]:
#Change the month datatype to make it easier to filter and move data
data['Month']= pd.to_datetime(data['Month'])
print(data.dtypes)
data.head()

Account_Number                        int64
Month                        datetime64[ns]
Product                              object
FICO_Score_Original                   int64
FICO_Score_Current                    int64
Commitments                         float64
Outstandings                        float64
LTV_Original                        float64
LTV_Current                         float64
Lien_Position                        object
Day_Past_Due                          int64
Interest_Variability_Code            object
Origination_DTI                     float64
APR                                 float64
Zip_Code                            float64
Total_Mortgage_Balances               int64
Total_HE_Balances                     int64
Num_Trades_Past_Due                 float64
Num_HE_Trades                       float64
dtype: object


,Account_Number,Month,Product,FICO_Score_Original,FICO_Score_Current,Commitments,Outstandings,LTV_Original,LTV_Current,Lien_Position,Day_Past_Due,Interest_Variability_Code,Origination_DTI,APR,Zip_Code,Total_Mortgage_Balances,Total_HE_Balances,Num_Trades_Past_Due,Num_HE_Trades
6,4999,2017-03-31,HELOAN,840,722,3.0,3.0,0.8,0.490,SR_LIEN,0,FXD,0.2,0.04,48162.0,0,133829,93.0,3.0
7,4999,2017-04-30,HELOAN,840,726,3.0,3.0,0.8,0.447,SR_LIEN,0,FXD,0.2,0.04,48162.0,0,133829,93.0,3.0
8,4999,2017-05-31,HELOAN,840,712,3.0,3.0,0.8,0.447,SR_LIEN,0,FXD,0.2,0.04,48162.0,0,133829,93.0,3.0
9,4999,2017-06-30,HELOAN,840,688,3.0,3.0,0.8,0.462,SR_LIEN,0,FXD,0.2,0.04,48162.0,0,133829,93.0,3.0
10,4999,2017-07-31,HELOAN,840,675,3.0,3.0,0.8,0.549,SR_LIEN,0,FXD,0.2,0.04,48162.0,0,133829,93.0,3.0


In [8]:
#Create datasets filtered by month to prepare to merge all account information for each month into one observation
mar = "2017-03-31"
apr = "2017-04-30"
may = "2017-05-31"
jun = "2017-06-30"
jul = "2017-07-31"


march = data[data['Month'] == mar] 
march = march.drop(['Month','Product'],axis=1)
march = march.rename(columns={'FICO_Score_Current': 'Mar_FICO','Commitments':'Mar_Commitments', 'Outstandings': 'Mar_Outstandings', 'LTV_Current':'Mar_LTV',
                             'Day_Past_Due':'Mar_Days_Past_Due'})



april = data[data['Month'] == apr] 
april = april.drop(['Month','Product','FICO_Score_Original','LTV_Original','Lien_Position', 'Interest_Variability_Code','Origination_DTI','APR','Zip_Code','Total_Mortgage_Balances',
                   'Total_HE_Balances','Num_Trades_Past_Due','Num_HE_Trades'],axis=1)
april = april.rename(columns={'FICO_Score_Current': 'Apr_FICO','Commitments':'Apr_Commitments', 'Outstandings': 'Apr_Outstandings', 'LTV_Current':'Apr_LTV',
                             'Day_Past_Due':'Apr_Days_Past_Due'})
 

may = data[data['Month'] == may] 
may = may.drop(['Month','Product','FICO_Score_Original','LTV_Original','Lien_Position', 'Interest_Variability_Code','Origination_DTI','APR','Zip_Code','Total_Mortgage_Balances',
                   'Total_HE_Balances','Num_Trades_Past_Due','Num_HE_Trades'],axis=1)
may = may.rename(columns={'FICO_Score_Current': 'May_FICO','Commitments':'May_Commitments', 'Outstandings': 'May_Outstandings', 'LTV_Current':'May_LTV',
                             'Day_Past_Due':'May_Days_Past_Due'})



june = data[data['Month'] == jun] 
june = june.drop(['Month','Product','FICO_Score_Original','LTV_Original','Lien_Position', 'Interest_Variability_Code','Origination_DTI','APR','Zip_Code','Total_Mortgage_Balances',
                   'Total_HE_Balances','Num_Trades_Past_Due','Num_HE_Trades'],axis=1)
june = june.rename(columns={'FICO_Score_Current': 'Jun_FICO','Commitments':'Jun_Commitments', 'Outstandings': 'Jun_Outstandings', 'LTV_Current':'Jun_LTV',
                             'Day_Past_Due':'Jun_Days_Past_Due'})

july = data[data['Month'] == jul] 
july = july.drop(['Month','Product','FICO_Score_Original','LTV_Original','Lien_Position', 'Interest_Variability_Code','Origination_DTI','APR','Zip_Code','Total_Mortgage_Balances',
                   'Total_HE_Balances','Num_Trades_Past_Due','Num_HE_Trades'],axis=1)
july = july.rename(columns={'FICO_Score_Current': 'Jul_FICO','Commitments':'Jul_Commitments', 'Outstandings': 'Jul_Outstandings', 'LTV_Current':'Jul_LTV',
                             'Day_Past_Due':'Jul_Days_Past_Due'})


In [9]:
#Merge month datasets
d = pd.merge(march, april, on="Account_Number")
d1 = pd.merge(d, may, on="Account_Number")
d = pd.merge(d1, june, on="Account_Number")
d1 = pd.merge(d, july, on="Account_Number")
d1.head()

,Account_Number,FICO_Score_Original,Mar_FICO,Mar_Commitments,Mar_Outstandings,LTV_Original,Mar_LTV,Lien_Position,Mar_Days_Past_Due,Interest_Variability_Code,...,Jun_FICO,Jun_Commitments,Jun_Outstandings,Jun_LTV,Jun_Days_Past_Due,Jul_FICO,Jul_Commitments,Jul_Outstandings,Jul_LTV,Jul_Days_Past_Due
0,4999,840,722,3.0,3.0,0.8,0.490,SR_LIEN,0,FXD,...,688,3.0,3.0,0.462,0,675,3.0,3.0,0.549,0
1,4998,732,682,2.0,2.0,0.9,0.879,SR_LIEN,0,FXD,...,651,2.0,2.0,0.879,0,637,2.0,2.0,0.862,0
2,4997,735,807,1.0,1.0,0.5,0.236,SR_LIEN,0,FXD,...,791,1.0,1.0,0.239,0,791,1.0,1.0,0.200,0
3,4996,818,799,1.0,1.0,0.3,0.032,JR_LIEN,0,FXD,...,789,1.0,1.0,0.023,0,797,1.0,1.0,0.020,0
4,4995,745,716,1.0,1.0,0.0,0.015,JR_LIEN,0,FXD,...,743,1.0,1.0,0.013,0,790,1.0,1.0,0.012,0


In [10]:
#Analyze datasets and drop zip code
print(d1.dtypes)
d1.drop('Zip_Code',axis=1)

Account_Number                 int64
FICO_Score_Original            int64
Mar_FICO                       int64
Mar_Commitments              float64
Mar_Outstandings             float64
LTV_Original                 float64
Mar_LTV                      float64
Lien_Position                 object
Mar_Days_Past_Due              int64
Interest_Variability_Code     object
Origination_DTI              float64
APR                          float64
Zip_Code                     float64
Total_Mortgage_Balances        int64
Total_HE_Balances              int64
Num_Trades_Past_Due          float64
Num_HE_Trades                float64
Apr_FICO                       int64
Apr_Commitments              float64
Apr_Outstandings             float64
Apr_LTV                      float64
Apr_Days_Past_Due              int64
May_FICO                       int64
May_Commitments              float64
May_Outstandings             float64
May_LTV                      float64
May_Days_Past_Due              int64
J

,Account_Number,FICO_Score_Original,Mar_FICO,Mar_Commitments,Mar_Outstandings,LTV_Original,Mar_LTV,Lien_Position,Mar_Days_Past_Due,Interest_Variability_Code,...,Jun_FICO,Jun_Commitments,Jun_Outstandings,Jun_LTV,Jun_Days_Past_Due,Jul_FICO,Jul_Commitments,Jul_Outstandings,Jul_LTV,Jul_Days_Past_Due
0,4999,840,722,3.0,3.0,0.8,0.490,SR_LIEN,0,FXD,...,688,3.0,3.0,0.462,0,675,3.0,3.0,0.549,0
1,4998,732,682,2.0,2.0,0.9,0.879,SR_LIEN,0,FXD,...,651,2.0,2.0,0.879,0,637,2.0,2.0,0.862,0
2,4997,735,807,1.0,1.0,0.5,0.236,SR_LIEN,0,FXD,...,791,1.0,1.0,0.239,0,791,1.0,1.0,0.200,0
3,4996,818,799,1.0,1.0,0.3,0.032,JR_LIEN,0,FXD,...,789,1.0,1.0,0.023,0,797,1.0,1.0,0.020,0
4,4995,745,716,1.0,1.0,0.0,0.015,JR_LIEN,0,FXD,...,743,1.0,1.0,0.013,0,790,1.0,1.0,0.012,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2307,1918,559,748,1.0,1.0,0.3,0.098,JR_LIEN,0,FXD,...,719,1.0,1.0,0.093,0,771,1.0,1.0,0.099,0
2308,1915,715,775,1.0,1.0,0.8,0.156,JR_LIEN,0,FXD,...,841,1.0,1.0,0.124,0,793,1.0,1.0,0.121,0
2309,1914,636,709,1.0,1.0,0.8,0.272,JR_LIEN,0,FXD,...,735,1.0,1.0,0.240,0,665,1.0,1.0,0.253,0
2310,1910,666,507,1.0,1.0,0.7,0.086,JR_LIEN,0,FXD,...,502,1.0,1.0,0.078,0,502,1.0,1.0,0.075,0


In [11]:
#Create categorical variables for Total_Mortgage_Balances, Total_HE_Balances, and each FICO Month to make the data
#Easier to read for regressions 

d1.loc[(d1['Total_Mortgage_Balances'].astype(int)==0) , 'Total_Mortgage_Balances'] = 0
d1.loc[(d1['Total_Mortgage_Balances'].astype(int) >0) & (d1['Total_Mortgage_Balances'].astype(int) <=50000), 'Total_Mortgage_Balances'] = 1
d1.loc[(d1['Total_Mortgage_Balances'].astype(int) >=50000 ) & (d1['Total_Mortgage_Balances'].astype(int) <=100000), 'Total_Mortgage_Balances'] = 2
d1.loc[(d1['Total_Mortgage_Balances'].astype(int) >=100000 ) & (d1['Total_Mortgage_Balances'].astype(int) <=200000), 'Total_Mortgage_Balances'] = 3
d1.loc[(d1['Total_Mortgage_Balances'].astype(int) >=200000 ) & (d1['Total_Mortgage_Balances'].astype(int) <=400000), 'Total_Mortgage_Balances'] = 4
d1.loc[(d1['Total_Mortgage_Balances'].astype(int) >=400000 ) & (d1['Total_Mortgage_Balances'].astype(int) <=600000), 'Total_Mortgage_Balances'] = 5
d1.loc[(d1['Total_Mortgage_Balances'].astype(int) >=600000 ) & (d1['Total_Mortgage_Balances'].astype(int) <=1000000), 'Total_Mortgage_Balances'] = 6
d1.loc[(d1['Total_Mortgage_Balances'].astype(int) >=1000000 ), 'Total_Mortgage_Balances'] = 7

d1.loc[(d1['Total_HE_Balances'].astype(int)==0) , 'Total_HE_Balances'] = 0
d1.loc[(d1['Total_HE_Balances'].astype(int) >0) & (d1['Total_HE_Balances'].astype(int) <=50000), 'Total_HE_Balances'] = 1
d1.loc[(d1['Total_HE_Balances'].astype(int) >=50000 ) & (d1['Total_HE_Balances'].astype(int) <=100000), 'Total_HE_Balances'] = 2
d1.loc[(d1['Total_HE_Balances'].astype(int) >=100000 ) & (d1['Total_HE_Balances'].astype(int) <=200000), 'Total_HE_Balances'] = 3
d1.loc[(d1['Total_HE_Balances'].astype(int) >=200000 ) & (d1['Total_HE_Balances'].astype(int) <=400000), 'Total_HE_Balances'] = 4
d1.loc[(d1['Total_HE_Balances'].astype(int) >=400000 ) & (d1['Total_HE_Balances'].astype(int) <=600000), 'Total_HE_Balances'] = 5
d1.loc[(d1['Total_HE_Balances'].astype(int) >=600000 ) & (d1['Total_HE_Balances'].astype(int) <=1000000), 'Total_HE_Balances'] = 6
d1.loc[(d1['Total_HE_Balances'].astype(int) >=1000000 ), 'Total_HE_Balances'] = "7"

d1.loc[(d1['FICO_Score_Original'].astype(int) <=580), 'FICO_Score_Original'] = 0 #Poor
d1.loc[(d1['FICO_Score_Original'].astype(int) >=580) & (d1['FICO_Score_Original'].astype(int) <=669), 'FICO_Score_Original'] = 1 #Fair
d1.loc[(d1['FICO_Score_Original'].astype(int) >=670) & (d1['FICO_Score_Original'].astype(int) <=739), 'FICO_Score_Original'] = 2 #Good
d1.loc[(d1['FICO_Score_Original'].astype(int) >=740) & (d1['FICO_Score_Original'].astype(int) <=799), 'FICO_Score_Original'] = 3 #Very Good
d1.loc[(d1['FICO_Score_Original'].astype(int) >=800), 'FICO_Score_Original'] = 4 #Exceptional

d1.loc[(d1['Mar_FICO'].astype(int) <=580), 'Mar_FICO'] = 0 #Poor
d1.loc[(d1['Mar_FICO'].astype(int) >=580) & (d1['Mar_FICO'].astype(int) <=669), 'Mar_FICO'] = 1 #Fair
d1.loc[(d1['Mar_FICO'].astype(int) >=670) & (d1['Mar_FICO'].astype(int) <=739), 'Mar_FICO'] = 2 #Good
d1.loc[(d1['Mar_FICO'].astype(int) >=740) & (d1['Mar_FICO'].astype(int) <=799), 'Mar_FICO'] = 3 #Very Good
d1.loc[(d1['Mar_FICO'].astype(int) >=800), 'Mar_FICO'] = 4 #Exceptional

d1.loc[(d1['Apr_FICO'].astype(int) <=580), 'Apr_FICO'] = 0 #Poor
d1.loc[(d1['Apr_FICO'].astype(int) >=580) & (d1['Apr_FICO'].astype(int) <=669), 'Apr_FICO'] = 1 #Fair
d1.loc[(d1['Apr_FICO'].astype(int) >=670) & (d1['Apr_FICO'].astype(int) <=739), 'Apr_FICO'] = 2 #Good
d1.loc[(d1['Apr_FICO'].astype(int) >=740) & (d1['Apr_FICO'].astype(int) <=799), 'Apr_FICO'] = 3 #Very Good
d1.loc[(d1['Apr_FICO'].astype(int) >=800), 'Apr_FICO'] = 4 #Exceptional

d1.loc[(d1['May_FICO'].astype(int) <=580), 'May_FICO'] = 0 #Poor
d1.loc[(d1['May_FICO'].astype(int) >=580) & (d1['May_FICO'].astype(int) <=669), 'May_FICO'] = 1 #Fair
d1.loc[(d1['May_FICO'].astype(int) >=670) & (d1['May_FICO'].astype(int) <=739), 'May_FICO'] = 2 #Good
d1.loc[(d1['May_FICO'].astype(int) >=740) & (d1['May_FICO'].astype(int) <=799), 'May_FICO'] = 3 #Very Good
d1.loc[(d1['May_FICO'].astype(int) >=800), 'May_FICO'] = 4 #Exceptional

d1.loc[(d1['Jun_FICO'].astype(int) <=580), 'Jun_FICO'] = 0 #Poor
d1.loc[(d1['Jun_FICO'].astype(int) >=580) & (d1['Jun_FICO'].astype(int) <=669), 'Jun_FICO'] = 1 #Fair
d1.loc[(d1['Jun_FICO'].astype(int) >=670) & (d1['Jun_FICO'].astype(int) <=739), 'Jun_FICO'] = 2 #Good
d1.loc[(d1['Jun_FICO'].astype(int) >=740) & (d1['Jun_FICO'].astype(int) <=799), 'Jun_FICO'] = 3 #Very Good
d1.loc[(d1['Jun_FICO'].astype(int) >=800), 'Jun_FICO'] = 4 #Exceptional

d1.loc[(d1['Jul_FICO'].astype(int) <=580), 'Jul_FICO'] = 0 #Poor
d1.loc[(d1['Jul_FICO'].astype(int) >=580) & (d1['Jul_FICO'].astype(int) <=669), 'Jul_FICO'] = 1 #Fair
d1.loc[(d1['Jul_FICO'].astype(int) >=670) & (d1['Jul_FICO'].astype(int) <=739), 'Jul_FICO'] = 2 #Good
d1.loc[(d1['Jul_FICO'].astype(int) >=740) & (d1['Jul_FICO'].astype(int) <=799), 'Jul_FICO'] = 3 #Very Good
d1.loc[(d1['Jul_FICO'].astype(int) >=800), 'Jul_FICO'] = 4 #Exceptional


d1.loc[(d1['FICO_Score_Original'].astype(int) < d1['Apr_FICO'].astype(int)) | (d1['FICO_Score_Original'].astype(int) < d1['Apr_FICO'].astype(int)) | (d1['FICO_Score_Original'].astype(int) < d1['May_FICO'].astype(int)) | 
       (d1['FICO_Score_Original'].astype(int) < d1['Jun_FICO'].astype(int)) | (d1['FICO_Score_Original'].astype(int) < d1['Jul_FICO'].astype(int)) | (d1['FICO_Score_Original'].astype(int) < d1['Mar_FICO'].astype(int)), 'FICO_Change'] = 1 #Negative
d1.loc[(d1['FICO_Score_Original'].astype(int) > d1['Apr_FICO'].astype(int)) | (d1['FICO_Score_Original'].astype(int) > d1['Apr_FICO'].astype(int)) | (d1['FICO_Score_Original'].astype(int) > d1['May_FICO'].astype(int)) | 
       (d1['FICO_Score_Original'].astype(int) > d1['Jun_FICO'].astype(int)) | (d1['FICO_Score_Original'].astype(int) > d1['Jul_FICO'].astype(int)) | (d1['FICO_Score_Original'].astype(int) > d1['Mar_FICO'].astype(int)), 'FICO_Change'] = 2 
d1.loc[(d1['FICO_Score_Original'] == d1['Apr_FICO']) | (d1['FICO_Score_Original'] == d1['Apr_FICO']) | (d1['FICO_Score_Original'] == d1['May_FICO']) | 
       (d1['FICO_Score_Original'] == d1['Jun_FICO']) | (d1['FICO_Score_Original'] == d1['Jul_FICO']) | (d1['FICO_Score_Original'].astype(int) == d1['Mar_FICO'].astype(int)), 'FICO_Change'] = 0 
        
        
pd.set_option('display.max_columns', None)
d1.head()


,Account_Number,FICO_Score_Original,Mar_FICO,Mar_Commitments,Mar_Outstandings,LTV_Original,Mar_LTV,Lien_Position,Mar_Days_Past_Due,Interest_Variability_Code,Origination_DTI,APR,Zip_Code,Total_Mortgage_Balances,Total_HE_Balances,Num_Trades_Past_Due,Num_HE_Trades,Apr_FICO,Apr_Commitments,Apr_Outstandings,Apr_LTV,Apr_Days_Past_Due,May_FICO,May_Commitments,May_Outstandings,May_LTV,May_Days_Past_Due,Jun_FICO,Jun_Commitments,Jun_Outstandings,Jun_LTV,Jun_Days_Past_Due,Jul_FICO,Jul_Commitments,Jul_Outstandings,Jul_LTV,Jul_Days_Past_Due,FICO_Change
0,4999,4,2,3.0,3.0,0.8,0.490,SR_LIEN,0,FXD,0.2,0.04,48162.0,0,3,93.0,3.0,2,3.0,3.0,0.447,0,2,3.0,3.0,0.447,0,2,3.0,3.0,0.462,0,2,3.0,3.0,0.549,0,2.0
1,4998,2,2,2.0,2.0,0.9,0.879,SR_LIEN,0,FXD,0.2,0.06,46017.0,0,2,1.0,2.0,2,2.0,2.0,0.870,0,1,2.0,2.0,0.887,0,1,2.0,2.0,0.879,0,1,2.0,2.0,0.862,0,0.0
2,4997,2,4,1.0,1.0,0.5,0.236,SR_LIEN,0,FXD,0.4,0.04,46052.0,1,1,93.0,1.0,3,1.0,1.0,0.246,0,3,1.0,1.0,0.258,0,3,1.0,1.0,0.239,0,3,1.0,1.0,0.200,0,1.0
3,4996,4,3,1.0,1.0,0.3,0.032,JR_LIEN,0,FXD,0.2,0.04,45840.0,0,1,93.0,2.0,3,1.0,1.0,0.030,0,2,1.0,1.0,0.025,0,3,1.0,1.0,0.023,0,3,1.0,1.0,0.020,0,2.0
4,4995,3,2,1.0,1.0,0.0,0.015,JR_LIEN,0,FXD,0.5,0.04,45833.0,3,1,93.0,2.0,3,1.0,1.0,0.015,0,3,1.0,1.0,0.014,0,3,1.0,1.0,0.013,0,3,1.0,1.0,0.012,0,0.0


In [12]:
#Convert string data into categorical variables so that it can be regressed
d1.loc[(d1['Interest_Variability_Code']=='FXD'), 'Interest_Variability_Code'] = 1
d1.loc[(d1['Interest_Variability_Code']=='VAR'), 'Interest_Variability_Code'] = 2     
d1.loc[(d1['Interest_Variability_Code']=='ADJ'), 'Interest_Variability_Code'] = 3

d1.loc[(d1['Lien_Position']=='JR_LIEN'), 'Lien_Position'] = 1
d1.loc[(d1['Lien_Position']=='SR_LIEN'), 'Lien_Position'] = 2
d1.loc[(d1['Lien_Position']=='OT_LIEN'), 'Lien_Position'] = 3 

d1['Lien_Position'] = d1['Lien_Position'].astype(float)
d1['Interest_Variability_Code'] = d1['Interest_Variability_Code'].astype(float)
d1['Total_HE_Balances'] = d1['Total_HE_Balances'].astype(float)

In [13]:
#Look at datatypes and confirm all missing values are dropped
print(d1.dtypes)
d1 = d1.dropna()

Account_Number                 int64
FICO_Score_Original            int64
Mar_FICO                       int64
Mar_Commitments              float64
Mar_Outstandings             float64
LTV_Original                 float64
Mar_LTV                      float64
Lien_Position                float64
Mar_Days_Past_Due              int64
Interest_Variability_Code    float64
Origination_DTI              float64
APR                          float64
Zip_Code                     float64
Total_Mortgage_Balances        int64
Total_HE_Balances            float64
Num_Trades_Past_Due          float64
Num_HE_Trades                float64
Apr_FICO                       int64
Apr_Commitments              float64
Apr_Outstandings             float64
Apr_LTV                      float64
Apr_Days_Past_Due              int64
May_FICO                       int64
May_Commitments              float64
May_Outstandings             float64
May_LTV                      float64
May_Days_Past_Due              int64
J

In [14]:
#Create new variables to be able to analyze the affect of greatest amount of commitments, average LTV and different
#FICO scores for the regressions 
d1['Max_Commitments'] = d1[['Mar_Commitments', 'Apr_Commitments','May_Commitments','Jun_Commitments','Jul_Commitments']].max(axis=1)
d1['Max_Outstandings'] = d1[['Mar_Outstandings', 'Apr_Outstandings','May_Outstandings','Jun_Outstandings','Jul_Outstandings']].max(axis=1)

d1['Max_FICO'] = d1[['Mar_FICO', 'Apr_FICO','May_FICO','Jun_FICO','Jul_FICO']].max(axis=1)
d1['Min_FICO'] = d1[['Mar_FICO', 'Apr_FICO','May_FICO','Jun_FICO','Jul_FICO']].min(axis=1)
d1['Avg_LTV'] = round((d1['Mar_LTV']+d1['Apr_LTV']+d1['May_LTV']+d1['Jun_LTV']+d1['Jul_LTV'])/5,1)
d1['Total_Days_Past_Due'] = (d1['Mar_Days_Past_Due']+d1['Apr_Days_Past_Due']+d1['May_Days_Past_Due']+d1['Jun_Days_Past_Due']+d1['Jul_Days_Past_Due'])
d1.loc[(d1['Total_Days_Past_Due'].astype(int) ==0), 'Past_Due'] = 0
d1.loc[(d1['Total_Days_Past_Due'].astype(int) >0), 'Past_Due'] = 1
d1 = d1.drop(['Mar_Outstandings', 'Apr_Outstandings','May_Outstandings','Jun_Outstandings','Jul_Outstandings','Mar_Commitments', 'Apr_Commitments','May_Commitments','Jun_Commitments','Jul_Commitments', 'Mar_FICO', 'Apr_FICO','May_FICO','Jun_FICO','Jul_FICO',
             'Mar_LTV', 'Apr_LTV','May_LTV','Jun_LTV','Jul_LTV','Zip_Code'],axis=1)

In [15]:
#Analyze finalized dataset
d1.head()

,Account_Number,FICO_Score_Original,LTV_Original,Lien_Position,Mar_Days_Past_Due,Interest_Variability_Code,Origination_DTI,APR,Total_Mortgage_Balances,Total_HE_Balances,Num_Trades_Past_Due,Num_HE_Trades,Apr_Days_Past_Due,May_Days_Past_Due,Jun_Days_Past_Due,Jul_Days_Past_Due,FICO_Change,Max_Commitments,Max_Outstandings,Max_FICO,Min_FICO,Avg_LTV,Total_Days_Past_Due,Past_Due
0,4999,4,0.8,2.0,0,1.0,0.2,0.04,0,3.0,93.0,3.0,0,0,0,0,2.0,3.0,3.0,2,2,0.5,0,0.0
1,4998,2,0.9,2.0,0,1.0,0.2,0.06,0,2.0,1.0,2.0,0,0,0,0,0.0,2.0,2.0,2,1,0.9,0,0.0
2,4997,2,0.5,2.0,0,1.0,0.4,0.04,1,1.0,93.0,1.0,0,0,0,0,1.0,1.0,1.0,4,3,0.2,0,0.0
3,4996,4,0.3,1.0,0,1.0,0.2,0.04,0,1.0,93.0,2.0,0,0,0,0,2.0,1.0,1.0,3,2,0.0,0,0.0
4,4995,3,0.0,1.0,0,1.0,0.5,0.04,3,1.0,93.0,2.0,0,0,0,0,0.0,1.0,1.0,3,2,0.0,0,0.0


In [16]:
#Create summary statistics
d1.describe()

,Account_Number,FICO_Score_Original,LTV_Original,Lien_Position,Mar_Days_Past_Due,Interest_Variability_Code,Origination_DTI,APR,Total_Mortgage_Balances,Total_HE_Balances,Num_Trades_Past_Due,Num_HE_Trades,Apr_Days_Past_Due,May_Days_Past_Due,Jun_Days_Past_Due,Jul_Days_Past_Due,FICO_Change,Max_Commitments,Max_Outstandings,Max_FICO,Min_FICO,Avg_LTV,Total_Days_Past_Due,Past_Due
count,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.0,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000,2312.000000
mean,3645.896194,2.766436,0.555796,1.711073,3.458910,1.0,0.383521,0.056280,0.619377,1.685554,79.731401,1.829585,3.544550,3.804498,4.026817,4.385813,0.625000,1.480536,1.477076,2.853374,1.985727,0.339446,19.220588,0.081747
std,865.069004,0.944105,0.258497,0.453362,28.329719,0.0,0.766347,0.021058,1.221659,0.895780,32.030304,0.997803,29.296581,31.750690,34.551865,36.734631,0.872652,0.755515,0.753697,1.148305,1.008516,0.325654,158.341045,0.274039
min,1907.000000,0.000000,0.000000,1.000000,0.000000,1.0,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2938.500000,2.000000,0.300000,1.000000,0.000000,1.0,0.200000,0.040000,0.000000,1.000000,93.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,2.000000,1.000000,0.100000,0.000000,0.000000
50%,3753.500000,3.000000,0.600000,2.000000,0.000000,1.0,0.300000,0.050000,0.000000,1.000000,93.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,3.000000,2.000000,0.300000,0.000000,0.000000
75%,4386.250000,4.000000,0.800000,2.000000,0.000000,1.0,0.400000,0.070000,1.000000,2.000000,93.000000,2.000000,0.000000,0.000000,0.000000,0.000000,2.000000,2.000000,2.000000,4.000000,3.000000,0.500000,0.000000,0.000000
max,4999.000000,4.000000,1.300000,2.000000,637.000000,1.0,10.500000,0.120000,7.000000,7.000000,93.000000,9.000000,635.000000,679.000000,737.000000,746.000000,2.000000,6.000000,6.000000,4.000000,4.000000,3.300000,3434.000000,1.000000


In [17]:
#Testing to see if charecteristics affect number of days of deliquency. Testing with Max FICO

x = d1[['Origination_DTI','APR','Total_Mortgage_Balances',
        'Num_HE_Trades','Max_Commitments','Max_Outstandings',
        'Max_FICO','Avg_LTV','Lien_Position','Interest_Variability_Code','Total_HE_Balances']]   

#d1.loc[(d1['Total_Days_Past_Due'].astype(int) != 0), 'Log_Days'] = np.log(d1['Total_Days_Past_Due']) 
d1.loc[(d1['Total_Days_Past_Due'].astype(int) != 0), 'Log_Days'] = np.sqrt(d1['Total_Days_Past_Due'])
d1.loc[(d1['Total_Days_Past_Due'].astype(int) == 0), 'Log_Days'] = 0

#y = d1['Log_Days']
y = d1['Total_Days_Past_Due']

regr = linear_model.LinearRegression()
regr.fit(x, y)
model1 = sm.OLS(y, x).fit()
predictions = model1.predict(x) 
 
print_model = model1.summary()
print(print_model)

                             OLS Regression Results                            
Dep. Variable:     Total_Days_Past_Due   R-squared:                       0.054
Model:                             OLS   Adj. R-squared:                  0.050
Method:                  Least Squares   F-statistic:                     13.24
Date:                 Sun, 31 Oct 2021   Prob (F-statistic):           7.69e-23
Time:                         22:54:40   Log-Likelihood:                -14925.
No. Observations:                 2312   AIC:                         2.987e+04
Df Residuals:                     2301   BIC:                         2.994e+04
Df Model:                           10                                         
Covariance Type:             nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Origination_

In [18]:
#Check heteroskedasticity 
resid1 = model1.resid
bptest = sms.diagnostic.het_breuschpagan(resid1,model1.model.exog)
#white_test = sms.diagnostic.het_white(resid1,model1.model.exog)
#white_test = het_white(model1.resid,  model1.exog)
print(bptest)

(56.82276195789784, 1.4390874131185218e-08, 5.797733901333465, 1.1415420682685465e-08)


In [19]:
#Run a GML to adjust for heteroskedasticity 
import statsmodels.api as sm
exog, endog = sm.add_constant(x), y
mod = sm.GLM(endog, exog,
             family=sm.families.Poisson(link=sm.families.links.log))
res = mod.fit()

print(res.summary())

                  Generalized Linear Model Regression Results                  
Dep. Variable:     Total_Days_Past_Due   No. Observations:                 2312
Model:                             GLM   Df Residuals:                     2301
Model Family:                  Poisson   Df Model:                           10
Link Function:                     log   Scale:                          1.0000
Method:                           IRLS   Log-Likelihood:                -91402.
Date:                 Sun, 31 Oct 2021   Deviance:                   1.8170e+05
Time:                         22:54:40   Pearson chi2:                 2.01e+10
No. Iterations:                      8                                         
Covariance Type:             nonrobust                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Origination_

<ipython-input-19-bab554e585c6>:4: DeprecationWarning: Calling Family(..) with a link class as argument is deprecated.
Use an instance of a link class instead.
  family=sm.families.Poisson(link=sm.families.links.log))


In [20]:
#Test for past due to see if certain characteristics can predict deliquency, Test with Max FICO
y = d1['Past_Due']

regr = linear_model.LinearRegression()
regr.fit(x, y)
model1 = sm.OLS(y, x).fit()
predictions = model1.predict(x) 
 
print_model = model1.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:               Past_Due   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.129
Method:                 Least Squares   F-statistic:                     35.11
Date:                Sun, 31 Oct 2021   Prob (F-statistic):           2.58e-64
Time:                        22:54:40   Log-Likelihood:                -123.08
No. Observations:                2312   AIC:                             268.2
Df Residuals:                    2301   BIC:                             331.4
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Origination_DTI       

In [21]:
#Run a GML to adjust for heteroskedasticity 
import statsmodels.api as sm
exog, endog = sm.add_constant(x), y
mod = sm.GLM(endog, exog,
             family=sm.families.Poisson(link=sm.families.links.log))
res = mod.fit()

print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Past_Due   No. Observations:                 2312
Model:                            GLM   Df Residuals:                     2301
Model Family:                 Poisson   Df Model:                           10
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -538.92
Date:                Sun, 31 Oct 2021   Deviance:                       699.85
Time:                        22:54:40   Pearson chi2:                 2.11e+03
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Origination_DTI       

<ipython-input-21-bab554e585c6>:4: DeprecationWarning: Calling Family(..) with a link class as argument is deprecated.
Use an instance of a link class instead.
  family=sm.families.Poisson(link=sm.families.links.log))


In [26]:
#Test for past due to see if certain characteristics can predict deliquency. Testing with Min FICO
x = d1[['Origination_DTI','APR','Total_Mortgage_Balances',
        'Num_HE_Trades','Max_Commitments','Max_Outstandings',
        'Min_FICO','Avg_LTV','Lien_Position','Interest_Variability_Code','Total_HE_Balances']]   

#d1.loc[(d1['Total_Days_Past_Due'].astype(int) != 0), 'Log_Days'] = np.log(d1['Total_Days_Past_Due']) 
d1.loc[(d1['Total_Days_Past_Due'].astype(int) != 0), 'Log_Days'] = np.sqrt(d1['Total_Days_Past_Due'])
d1.loc[(d1['Total_Days_Past_Due'].astype(int) == 0), 'Log_Days'] = 0

#y = d1['Log_Days']
y = d1['Past_Due']

regr = linear_model.LinearRegression()
regr.fit(x, y)
model1 = sm.OLS(y, x).fit()
predictions = model1.predict(x) 
 
print_model = model1.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:               Past_Due   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.121
Method:                 Least Squares   F-statistic:                     32.73
Date:                Sun, 31 Oct 2021   Prob (F-statistic):           6.31e-60
Time:                        23:17:39   Log-Likelihood:                -133.48
No. Observations:                2312   AIC:                             289.0
Df Residuals:                    2301   BIC:                             352.2
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Origination_DTI       

In [27]:
#Run a GML to adjust for heteroskedasticity 
import statsmodels.api as sm
exog, endog = sm.add_constant(x), y
mod = sm.GLM(endog, exog,
             family=sm.families.Poisson(link=sm.families.links.log))
res = mod.fit()

print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Past_Due   No. Observations:                 2312
Model:                            GLM   Df Residuals:                     2301
Model Family:                 Poisson   Df Model:                           10
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -537.56
Date:                Sun, 31 Oct 2021   Deviance:                       697.12
Time:                        23:17:43   Pearson chi2:                 2.23e+03
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Origination_DTI       

<ipython-input-27-4584e0ea7239>:5: DeprecationWarning: Calling Family(..) with a link class as argument is deprecated.
Use an instance of a link class instead.
  family=sm.families.Poisson(link=sm.families.links.log))


In [28]:
#Test for past due to see if certain characteristics can predict deliquency. Testing with Original FICO

x = d1[['Origination_DTI','APR','Total_Mortgage_Balances',
        'Num_HE_Trades','Max_Commitments','Max_Outstandings',
        'FICO_Score_Original','Avg_LTV','Lien_Position','Interest_Variability_Code','Total_HE_Balances']]   

#d1.loc[(d1['Total_Days_Past_Due'].astype(int) != 0), 'Log_Days'] = np.log(d1['Total_Days_Past_Due']) 
d1.loc[(d1['Total_Days_Past_Due'].astype(int) != 0), 'Log_Days'] = np.sqrt(d1['Total_Days_Past_Due'])
d1.loc[(d1['Total_Days_Past_Due'].astype(int) == 0), 'Log_Days'] = 0

#y = d1['Log_Days']
y = d1['Past_Due']

regr = linear_model.LinearRegression()
regr.fit(x, y)
model1 = sm.OLS(y, x).fit()
predictions = model1.predict(x) 
 
print_model = model1.summary()
print(print_model)

                            OLS Regression Results                            
Dep. Variable:               Past_Due   R-squared:                       0.043
Model:                            OLS   Adj. R-squared:                  0.038
Method:                 Least Squares   F-statistic:                     10.25
Date:                Sun, 31 Oct 2021   Prob (F-statistic):           4.51e-17
Time:                        23:17:56   Log-Likelihood:                -236.88
No. Observations:                2312   AIC:                             495.8
Df Residuals:                    2301   BIC:                             559.0
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                                coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Origination_DTI       

In [29]:
#Run a GML to adjust for heteroskedasticity 
import statsmodels.api as sm
exog, endog = sm.add_constant(x), y
mod = sm.GLM(endog, exog,
             family=sm.families.Poisson(link=sm.families.links.log))
res = mod.fit()

print(res.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:               Past_Due   No. Observations:                 2312
Model:                            GLM   Df Residuals:                     2301
Model Family:                 Poisson   Df Model:                           10
Link Function:                    log   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -619.60
Date:                Sun, 31 Oct 2021   Deviance:                       861.20
Time:                        23:18:01   Pearson chi2:                 2.02e+03
No. Iterations:                     6                                         
Covariance Type:            nonrobust                                         
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Origination_DTI       

<ipython-input-29-4584e0ea7239>:5: DeprecationWarning: Calling Family(..) with a link class as argument is deprecated.
Use an instance of a link class instead.
  family=sm.families.Poisson(link=sm.families.links.log))
